# 17

In [1]:
# 런타임 3분 소요
!pip install transformers sentencepiece
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

# 모델 및 토크나이저 불러오기
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

#18

In [2]:
# 토크나이징을 통한 인코딩(입력문이 하나인 경우)
input = tokenizer.encode("I evaluated the performance of GPT-Neo developed by OpenAI.", return_tensors="pt")

# 첫번째 인코딩 결과(결과적으로 하나의 입력문장)를 확인
print(input[0])

# 첫번째 인코딩 결과를 디코딩
print(tokenizer.decode(input[0]))


tensor([   40, 16726,   262,  2854,   286,   402, 11571,    12,  8199,    78,
         4166,   416,  4946, 20185,    13])
I evaluated the performance of GPT-Neo developed by OpenAI.


In [3]:
# 토크나이징을 통한 인코딩(입력문이 복수인 경우)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

input = tokenizer.batch_encode_plus(["I evaluated the performance of GPT-Neo developed by OpenAI.","I evaluated the performance of GPT developed by OpenAI."], padding=True, truncation=True, return_tensors="pt")

# 인코딩 결과 확인
print(input['input_ids'])

# 디코딩
print([tokenizer.decode(input['input_ids'][i]) for i in range(len(input['input_ids']))])

tensor([[   40, 16726,   262,  2854,   286,   402, 11571,    12,  8199,    78,
          4166,   416,  4946, 20185,    13],
        [   40, 16726,   262,  2854,   286,   402, 11571,  4166,   416,  4946,
         20185,    13, 50257, 50257, 50257]])
['I evaluated the performance of GPT-Neo developed by OpenAI.', 'I evaluated the performance of GPT developed by OpenAI.[PAD][PAD][PAD]']


# 19

In [4]:
# 토크나이징
input = tokenizer.batch_encode_plus(["I evaluated the performance of GPT2 developed by OpenAI.", "Vaccine for new coronavirus in the UK","3.1415926535"], max_length=5, truncation=True, padding=True, return_tensors="pt")

In [5]:
# 인코딩 결과 확인
input['input_ids']

tensor([[   40, 16726,   262,  2854,   286],
        [   53,  4134,   500,   329,   649],
        [   18,    13,  1415, 19707, 22980]])

In [7]:
# 인코딩 결과를 model.generate( )에 투입
generated = model.generate(input['input_ids'])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [8]:
# model.generate( ) 결과 건수(len) 확인
len(generated)

3

In [9]:
# 디코딩
generated_text = tokenizer.batch_decode(generated)

# enumerate( )는 변수 generated_text에 있는 키값(value)에
# 0, 1, 2,...로 시작하는 숫자 키(key)를 부여
for i, sentence in enumerate(generated_text):
  print(f'No.{i+1}')
  print(f"{sentence}\n")


No.1
I evaluated the performance of the proposed method on the real-world dataset. The results are shown in

No.2
Vaccine for new-borns

The vaccine for new-borns is a vaccine

No.3
3.1415926535897932384626433832795028841971693



### 주의점
아래 코드를 참고

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")

# 패딩 토큰 추가
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# 첫 번째 입력 데이터 인코딩 (패딩 및 트렁케이션 설정)
inputs1 = tokenizer.batch_encode_plus(
    ["I evaluated the performance of GPT-Neo developed by OpenAI.",
     "I evaluated the performance of GPT developed by OpenAI."],
    padding=True,
    truncation=True,
    return_tensors="pt"
)

# 두 번째 입력 데이터 인코딩 (최대 길이 설정 및 패딩)
inputs2 = tokenizer.batch_encode_plus(
    ["I evaluated the performance of GPT2 developed by OpenAI.",
     "Vaccine for new coronavirus in the UK",
     "3.1415926535"],
    max_length=5,
    truncation=True,
    padding=True,
    return_tensors="pt"
)

# 어텐션 마스크를 모델에 전달하여 생성
generated = model.generate(input_ids=inputs2['input_ids'], attention_mask=inputs2['attention_mask'])

# 생성된 텍스트 디코딩
decoded_outputs = tokenizer.batch_decode(generated, skip_special_tokens=True)

# 결과 출력
for i, text in enumerate(decoded_outputs):
    print(f"Generated text {i+1}: {text}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text 1: I evaluated the performance of the proposed method on the real-world dataset. The results are shown in
Generated text 2: Vaccine for new-borns

The vaccine for new-borns is a vaccine
Generated text 3: 3.1415926535897932384626433832795028841971693


In [12]:
from transformers import AutoTokenizer

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# 텍스트를 토크나이저로 인코딩
encoded_input = tokenizer.encode("Hello, how are you?", add_special_tokens=True)
print(f"Encoded input: {encoded_input}")

# 디코딩 시 특수 토큰 포함 여부에 따른 출력
decoded_output_with_special_tokens = tokenizer.decode(encoded_input)
decoded_output_without_special_tokens = tokenizer.decode(encoded_input, skip_special_tokens=True)

print(f"Decoded output with special tokens: {decoded_output_with_special_tokens}")
print(f"Decoded output without special tokens: {decoded_output_without_special_tokens}")

Encoded input: [15496, 11, 703, 389, 345, 30]
Decoded output with special tokens: Hello, how are you?
Decoded output without special tokens: Hello, how are you?


In [15]:
from transformers import AutoTokenizer

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# 특수 토큰 확인
print(f"Special tokens: {tokenizer.special_tokens_map}")

# 인코딩 및 디코딩 예제
encoded_input = tokenizer.encode("Hello, how are you?", add_special_tokens=True)
print(f"Encoded input: {encoded_input}")

# 디코딩 시 특수 토큰 포함 여부에 따른 출력
decoded_output_with_special_tokens = tokenizer.decode(encoded_input)
decoded_output_without_special_tokens = tokenizer.decode(encoded_input, skip_special_tokens=True)

print(f"Decoded output with special tokens: {decoded_output_with_special_tokens}")
print(f"Decoded output without special tokens: {decoded_output_without_special_tokens}")

Special tokens: {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}
Encoded input: [15496, 11, 703, 389, 345, 30]
Decoded output with special tokens: Hello, how are you?
Decoded output without special tokens: Hello, how are you?


In [16]:
from transformers import AutoTokenizer

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# 특수 토큰 추가
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'sep_token': '[SEP]', 'cls_token': '[CLS]'})

# 특수 토큰 확인
print(f"Special tokens: {tokenizer.special_tokens_map}")

# 텍스트 인코딩
encoded_input = tokenizer.encode("Hello, how are you?", add_special_tokens=True)
print(f"Encoded input: {encoded_input}")

# 디코딩 시 특수 토큰 포함 여부에 따른 출력
decoded_output_with_special_tokens = tokenizer.decode(encoded_input)
decoded_output_without_special_tokens = tokenizer.decode(encoded_input, skip_special_tokens=True)

print(f"Decoded output with special tokens: {decoded_output_with_special_tokens}")
print(f"Decoded output without special tokens: {decoded_output_without_special_tokens}")

Special tokens: {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]'}
Encoded input: [15496, 11, 703, 389, 345, 30]
Decoded output with special tokens: Hello, how are you?
Decoded output without special tokens: Hello, how are you?


In [17]:
from transformers import AutoTokenizer

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# 특수 토큰 추가
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'sep_token': '[SEP]', 'cls_token': '[CLS]'})

# 특수 토큰 확인
print(f"Special tokens: {tokenizer.special_tokens_map}")

# 특수 토큰을 직접 포함하여 텍스트 인코딩
text_with_special_tokens = "[CLS] Hello, how are you? [SEP]"
encoded_input = tokenizer.encode(text_with_special_tokens, add_special_tokens=False)
print(f"Encoded input: {encoded_input}")

# 디코딩 시 특수 토큰 포함 여부에 따른 출력
decoded_output_with_special_tokens = tokenizer.decode(encoded_input)
decoded_output_without_special_tokens = tokenizer.decode(encoded_input, skip_special_tokens=True)

print(f"Decoded output with special tokens: {decoded_output_with_special_tokens}")
print(f"Decoded output without special tokens: {decoded_output_without_special_tokens}")

Special tokens: {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]'}
Encoded input: [50259, 18435, 11, 703, 389, 345, 30, 220, 50258]
Decoded output with special tokens: [CLS] Hello, how are you? [SEP]
Decoded output without special tokens:  Hello, how are you? 


In [14]:
from transformers import AutoTokenizer

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 텍스트를 특수 토큰과 함께 인코딩
encoded_input = tokenizer.encode("Hello, how are you?", add_special_tokens=True)
print(f"Encoded input: {encoded_input}")

# 특수 토큰을 포함하여 디코딩
decoded_output_with_special_tokens = tokenizer.decode(encoded_input)
print(f"Decoded output with special tokens: {decoded_output_with_special_tokens}")

# 특수 토큰을 제외하여 디코딩
decoded_output_without_special_tokens = tokenizer.decode(encoded_input, skip_special_tokens=True)
print(f"Decoded output without special tokens: {decoded_output_without_special_tokens}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Encoded input: [101, 7592, 1010, 2129, 2024, 2017, 1029, 102]
Decoded output with special tokens: [CLS] hello, how are you? [SEP]
Decoded output without special tokens: hello, how are you?


```
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

# 토크나이저와 모델 로드
model_name = "llama-2-model-name"  # 실제 모델 이름으로 변경
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# pad_token을 eos_token으로 설정
tokenizer.pad_token = tokenizer.eos_token

# 데이터셋 준비 (예시 데이터셋 사용)
train_texts = ["Hello, how are you?", "I am fine, thank you.", "How about you?"]
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors="pt")

# 트레이닝 인수 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    logging_dir='./logs',
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
)

# 모델 파인튜닝
trainer.train()
```

# 20

In [ ]:
# 코랩 메모리가 clear된 경우만 transformers 재설치
# !pip install transformers

# 모델 및 토크나이저 불러오기
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

model = AutoModelWithLMHead.from_pretrained("distilgpt2")


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1362: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


# 21

In [18]:
# 토크나이징. 출력은 파이토치 텐서(pt)로 받음
input_ids = tokenizer.encode("I like gpt because it's", return_tensors='pt')

# max-length 값을 12로 설정
greedy_output = model.generate(input_ids, max_length=12)

# *표시를 100번 실행하여 *로 구성된 줄 만들기
print("Output:\n" + 100 * '-')

# 인코딩 결과 디코딩
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I like gpt because it's a good way to get


## 22

In [19]:
# 모델 및 토크나어지 불러오기
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-small")

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1712: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# 23

In [20]:
# 토크나이징
input_ids = tokenizer.encode("I like gpt because it's", return_tensors='pt')

# max-length 값을 30으로 설정
greedy_output = model.generate(input_ids, max_length=30)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I like gpt because it's a good way to get a feel for the game.


In [21]:
# 보너스 코딩: 다른 예문으로 추가 코딩
# 토크나이징
input_ids = tokenizer.encode("Covid19 delta is spreading", return_tensors='pt')

# max-length 값을 50으로 설정
greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
Covid19 delta is spreading the word


## 24

In [ ]:
# 코랩 메모리가 clear된 경우만 transformers 재설치
!pip install transformers

In [22]:
# transformers 라이브러리에서 pipeline 불러오기
from transformers import pipeline

# pipeline에 과업(fill-mask) 및 모델 지정
unmasker = pipeline('fill-mask', model='bert-base-uncased')

# pipeline을 인스턴스화한 변수 unmasker에 [MASK] 토큰을 지닌 입력문장 투입
unmasker("MLM and NSP is the [MASK] task of BERT.")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.2572782337665558,
  'token': 2364,
  'token_str': 'main',
  'sequence': 'mlm and nsp is the main task of bert.'},
 {'score': 0.20740699768066406,
  'token': 3078,
  'token_str': 'primary',
  'sequence': 'mlm and nsp is the primary task of bert.'},
 {'score': 0.06773315370082855,
  'token': 2034,
  'token_str': 'first',
  'sequence': 'mlm and nsp is the first task of bert.'},
 {'score': 0.06548502296209335,
  'token': 2430,
  'token_str': 'central',
  'sequence': 'mlm and nsp is the central task of bert.'},
 {'score': 0.0616740919649601,
  'token': 3937,
  'token_str': 'basic',
  'sequence': 'mlm and nsp is the basic task of bert.'}]

In [29]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

# 텍스트 인코딩 및 마스킹
inputs = tokenizer("We [MASK] Dragon soup.", return_tensors="pt")

# 모델 예측
with torch.no_grad():
    outputs = model(**inputs)

# 마스킹된 토큰의 로짓(logits) 추출
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = outputs.logits[0, mask_token_index, :]

# 소프트맥스 확률 계산
softmax = torch.nn.functional.softmax(mask_token_logits, dim=-1)
top_5_tokens = torch.topk(softmax, 5, dim=-1)

# 출력
for token_id, score in zip(top_5_tokens.indices[0], top_5_tokens.values[0]):
    print(f"Token: {tokenizer.decode([token_id])}, Score: {score.item()}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Token: had, Score: 0.3979828655719757
Token: ate, Score: 0.19590364396572113
Token: ordered, Score: 0.06718766689300537
Token: made, Score: 0.05402311682701111
Token: eat, Score: 0.041049499064683914


### BERT의 [CLS] 토큰
[CLS] 토큰은 BERT와 같은 트랜스포머 모델에서 문장이나 문단 수준의 정보를 요약하는 데 사용되며 BERT는 입력 시퀀스의 첫 번째 위치에 [CLS] 토큰을 추가하고, 이 토큰의 출력 벡터를 사용하여 문장 수준의 다양한 작업을 수행합니다. [CLS] 토큰은 특히 다음과 같은 작업에서 유용:

1.문장 분류 (Sentiment Analysis, Text Classification)

문장의 감정 분석이나 주제 분류와 같은 작업에서 [CLS] 토큰의 출력 벡터를 사용하여 문장을 하나의 클래스(예: 긍정, 부정)로 분류.

2.문장 간 관계 예측 (Next Sentence Prediction)

BERT의 사전 훈련 작업 중 하나인 NSP(Next Sentence Prediction)에서는 두 문장이 주어졌을 때 두 번째 문장이 첫 번째 문장의 다음 문장인지 여부를 예측. 이 작업에서도 [CLS] 토큰의 출력 벡터를 사용.

3.문장 임베딩 (Sentence Embedding)

문장을 고정된 길이의 벡터로 변환하는 데 [CLS] 토큰의 출력 벡터를 사용. 이는 문장 간 유사도를 계산하거나, 문장을 입력으로 받는 다른 모델의 입력으로 사용할 수 있음.

4.QA 시스템에서의 문장 수준 정보
질의응답(QA) 시스템에서 [CLS] 토큰의 벡터는 질문과 문맥 간의 관련성을 평가하는 데 사용.

### [정리]
[CLS] 토큰은 입력 시퀀스의 첫 번째 토큰으로, 전체 시퀀스의 정보를 요약.  
이 토큰의 출력 벡터는 문장 분류, 문장 간 관계 예측, 문장 임베딩 등 다양한 작업에서 사용.  
BERT와 같은 모델에서 [CLS] 토큰의 벡터를 추출하여 여러 NLP 작업을 효과적으로 수행.


In [31]:
from transformers import BertTokenizer, BertModel
import torch

# BERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# 텍스트 인코딩
inputs = tokenizer("This is a sample sentence.", return_tensors="pt")

# 모델 예측
outputs = model(**inputs)

# [CLS] 토큰의 출력 벡터 추출
cls_output = outputs.last_hidden_state[:, 0, :]  # 배치의 첫 번째 토큰([CLS])의 출력

# [CLS] 벡터를 사용하여 분류 작업 수행 (예: 로지스틱 회귀)
# 여기서는 예시로 간단히 출력 벡터를 출력합니다.
print(cls_output)

tensor([[-1.9931e-01, -2.1006e-01, -1.9500e-01, -3.3238e-01, -5.2126e-01,
         -3.7337e-01,  1.5419e-01,  4.0006e-01,  3.5362e-02, -1.5012e-01,
         -3.5267e-01, -2.2861e-01, -9.4948e-02,  1.0328e-01,  5.1352e-01,
         -9.0901e-02,  2.3864e-01,  4.8223e-01,  3.1379e-01, -4.0566e-01,
          9.8626e-02, -8.8767e-02, -2.7461e-01, -5.0871e-01,  2.0913e-01,
         -3.0627e-01, -4.2952e-02, -4.1310e-01, -4.0709e-02,  8.0825e-02,
         -6.0292e-02,  3.5720e-01, -3.4175e-01, -1.4824e-01,  3.9080e-01,
         -1.3152e-01,  3.6742e-01, -9.1255e-03,  4.3404e-01,  1.4182e-01,
         -2.6785e-01, -8.7463e-02,  2.6639e-01,  5.4087e-02, -5.5893e-02,
         -5.7430e-01, -2.8187e+00, -3.4313e-01, -2.7937e-01, -3.0941e-01,
         -8.8544e-02, -2.2791e-01,  4.0630e-01,  5.0902e-01, -2.2796e-01,
          3.7730e-01, -2.9018e-01,  3.1223e-01,  1.9454e-01, -1.0551e-01,
          1.5179e-01, -6.6327e-02, -5.1237e-02, -1.4962e-01, -1.7307e-01,
          6.6952e-01, -1.2911e-01,  3.

In [34]:
from transformers import BertTokenizer, BertModel
import torch
import torch.nn.functional as F
import torch.nn as nn

# BERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# 간단한 분류기 정의 (BERT의 [CLS] 출력 벡터를 입력으로 받음)
class SimpleClassifier(nn.Module):
    def __init__(self, hidden_size, num_classes):
        super(SimpleClassifier, self).__init__()
        self.linear = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        return self.linear(x)

# 분류기 인스턴스화
num_classes = 2  # 예: 긍정/부정 분류
classifier = SimpleClassifier(model.config.hidden_size, num_classes)

# 텍스트 인코딩
inputs = tokenizer("This is a positive sentence.", return_tensors="pt")

# 모델 예측
outputs = model(**inputs)

# [CLS] 토큰의 출력 벡터 추출
cls_output = outputs.last_hidden_state[:, 0, :]  # 배치의 첫 번째 토큰([CLS])의 출력

# 분류기 출력
logits = classifier(cls_output)
print(logits)

probabilities = F.softmax(logits, dim=-1)
print(probabilities)

tensor([[-0.2943,  0.1966]], grad_fn=<AddmmBackward0>)
tensor([[0.3797, 0.6203]], grad_fn=<SoftmaxBackward0>)


# 25

In [35]:
from transformers import pipeline

# 모델명이 바뀌었음에 유의
unmasker = pipeline('fill-mask', model='distilbert-base-uncased')
unmasker("MLM and NSP is the [MASK] task of BERT.")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[{'score': 0.259024053812027,
  'token': 3078,
  'token_str': 'primary',
  'sequence': 'mlm and nsp is the primary task of bert.'},
 {'score': 0.16309823095798492,
  'token': 2364,
  'token_str': 'main',
  'sequence': 'mlm and nsp is the main task of bert.'},
 {'score': 0.08182742446660995,
  'token': 4563,
  'token_str': 'core',
  'sequence': 'mlm and nsp is the core task of bert.'},
 {'score': 0.04023803398013115,
  'token': 7037,
  'token_str': 'dual',
  'sequence': 'mlm and nsp is the dual task of bert.'},
 {'score': 0.02484491467475891,
  'token': 4054,
  'token_str': 'principal',
  'sequence': 'mlm and nsp is the principal task of bert.'}]

# 26

In [36]:
from transformers import pipeline

# 모델명이 바뀌었음에 유의
unmasker = pipeline('fill-mask', model='albert-base-v2')
unmasker("mlm and nsp is the [MASK] task of bert.")

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForMaskedLM: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

[{'score': 0.04760090634226799,
  'token': 6612,
  'token_str': 'ultimate',
  'sequence': 'mlm and nsp is the ultimate task of bert.'},
 {'score': 0.024472208693623543,
  'token': 20766,
  'token_str': 'hardest',
  'sequence': 'mlm and nsp is the hardest task of bert.'},
 {'score': 0.023495132103562355,
  'token': 1256,
  'token_str': 'primary',
  'sequence': 'mlm and nsp is the primary task of bert.'},
 {'score': 0.021575216203927994,
  'token': 407,
  'token_str': 'main',
  'sequence': 'mlm and nsp is the main task of bert.'},
 {'score': 0.018088074401021004,
  'token': 18369,
  'token_str': 'foremost',
  'sequence': 'mlm and nsp is the foremost task of bert.'}]